# Créer et exporter une catégorie au format Moodle XML

Besoin de 
`pip install array-to-latex` et `pip install xmltodict`

Si vous ne savez pas comment faire, décommentez la case suivante et exécutez le code (prend une bonne minute), ça devrait marcher

In [124]:
#import sys
#!{sys.executable} -m pip install xmltodict
#!{sys.executable} -m pip install array-to-latex

In [3]:
# Permet a une cellule d'avoir plus d'un display en sortie
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Technical functions:

In [6]:
from xml.dom.minidom import parseString
from xml.sax.saxutils import unescape
import xmltodict
import json
import io

In [57]:
def savestr(string, filename="new.txt", raw=False):
    if raw:
        string = string.replace('\t','') # no tabs
        string = string.replace('\n','') # no linebreak
    else:
        string = string.replace('\t','  ') # double space instead of tabs
    text_file = io.open(filename, "w", encoding='utf8') # essential for accents and other characters
    text_file.write(string)
    text_file.close()
    
def html(string):
    if string is "":
        return string
    else:
        return "<![CDATA[<p>\(\)" + string + "</p>]]>"  # \(\) pour activer latex dans Moodle

In [8]:
def set_oparg(name, default_value, **opargs): #optional argument manager
    if name in opargs:
        return opargs.get(name)
    else:
        return default_value

### Création de Moodle XML

In [84]:
class Question():
    """ 
        Object collecting the parameters of a question under the form of a dictionary.
        Methods:
        _set(field, value) : e.g. _set("questiontext", "What is $2+2$?")
    """
    def __init__(self, value="Default question"): # optional value for questiontext
        # DIRTY part where we gather all we need about the question
        self.structure = { 
            '@type': {'default': 'essay', 'depth': 0},
            "name": {'default': "Default question title", 'depth': 1},
            "questiontext": {'default': value, 'depth': 1, 'html': True},
            "generalfeedback": {'default': "", 'depth': 1, 'html': True},
            "defaultgrade": {'default': 1.0, 'depth': 0},
            "penalty": {'default': 0.0, 'depth': 0},
            "hidden": {'default': 0, 'depth': 0},
            "idnumber": {'default': "", 'depth': 0},
            "responseformat": {'default': "editor", 'depth': 0},
            "responserequired": {'default': 1, 'depth': 0},
            "responsefieldlines": {'default': 10, 'depth': 0},
            "attachments": {'default': 0, 'depth': 0},
            "attachmentsrequired": {'default': 0, 'depth': 0},
            "graderinfo": {'default': "", 'depth': 1, 'html': True},
            "responsetemplate": {'default': "", 'depth': 0, 'html': True},
        }
        # The proper question in a dictionary ready to turn into xml
        self.dict = {}
        for field in self.structure:
            self._set(field, self.structure[field]['default'])
               
    def _set(self, field, value=""):
        """ Assigns a value to a field of a Question """
        if self.structure[field]['depth'] == 0: 
            self.dict[field] = value
        elif self.structure[field]['depth'] == 1:
            if ('html' in self.structure[field]) and self.structure[field]['html']:
                self.dict[field] = {"@format": "html", "text": html(value)}
            else: # no html
                self.dict[field] = {"text": value}

In [104]:
class Category():
    """ 
        Object collecting Questions under the form of a category, ready to export to Moodle.
        Methods:
        _set(name, description) : e.g. _set("my_category", "list of questions about ... ")
        append(question) : adds a Question to the Category
        save(file_name) : save the Category into Moodle-XML
    """
    def __init__(self, name="Default category name", description=""):
        self.dict = { "quiz": { "question": [{}] } }
        self.questions = self.dict["quiz"]["question"]
        self._set(name, description)
    
    def _set(self, name="Default category name", description=""):
        qcat = {
            "@type": "category",
            "category": {"text": "$module$/top/" + name},
            "info": {"@format": "html", "text": html(description)}
            }
        self.questions[0] = qcat
        
    def append(self, question): # adds a Question to a Category
        self.questions.append(question.dict)
        
    def save(self, file_name="default_name"):
        """ Save a category under the format Moodle XML """
        category_xml = xmltodict.unparse(self.dict, pretty=True)
        savestr(unescape(category_xml), file_name + ".xml")

### Exemple : Générer une question dans une catégorie et les envoyer dans Moodle

In [111]:
category_name = "ma super catégorie"
category = Category(category_name)

In [127]:
questiontext = " Calculer la dérivée de $f(x) = e^x + \frac{1}{2} \Vert x \Vert^2$."
title = "Question de cours (dérivée)" # optional
correction = "$e^x + x$" # optional

question = Question(questiontext) # creates the question
question._set("name", title) # adds some optional information
question._set("graderinfo", correction)
category.append(question) # ajoute la question à la catégorie

category.save("ma_categorie") # crée un fichier déguelasse mais prêt à l'export dans Moodle

In [128]:
# une deuxième question pour la route ?
question = Question()
question._set("questiontext", "Combien font $2+2$?")
question._set("name", "Question d'arithmétique")
question._set("graderinfo", "Réponse : $\sqrt{16}$")

category.append(question)
category.save("ma_categorie") # maintenant on a deux questions dans la catégorie

La dernière commande crée un fichier `ma_categorie.xml` qu'il suffit d'importer dans Moodle.
Dans l'interface du cours, à droite, aller dans `banque de questions > importer ` puis 
- format de fichier : Format XML Moodle
- Généraux : Cocher "Obtenir la catégorie à partir du fichier", "Obtenir le contexte à partir du fichier" si ce n'est déjà fait
- Glisser le fichier
- Cliquer "importation"
- Si tout est bon (vert) valider encore (continuer).
Ensuite les questions sont prêtes à l'utilisation dans tous les tests.


### Exemple : Générer plein de questions aléatoirement et les envoyer dans Moodle

Ici on montre ce qu'il est possible de faire, évidemment chacun doit adapter à ses besoins.

In [116]:
import numpy as np
from numpy.random import randint
import array_to_latex as a2l

In [117]:
# On veut générer des matrices entières aléatoires et récupérer leur code latex
def strlatex(A): 
    return a2l.to_ltx(A, frmt = '{:2.0f}', arraytype = 'pmatrix', print_out=False)
A = randint(-6,6,(3,3))
print(strlatex(A))

\begin{pmatrix}
 -1  & -1  & -1 \\
 -6  &   3  &   4 \\
   2  &   3  & -2 
\end{pmatrix}


In [126]:
# du coup on peut en mettre plein les questions
category = Category(name="produit matriciel", 
                    description="liste de questions sur le produit matriciel")
for k in range(5):
    A = randint(-6,6,(3,3))
    B  = randint(-6,6,(3,2))
    latex = "Calculer le produit matriciel suivant : $$" + strlatex(A) + strlatex(B) + " $$"
    question = Question(latex)
    question._set("name", "Produit matriciel")
    question._set("graderinfo", strlatex(A@B)) # python nous donne la réponse 
    category.append(question)
    
category.save("ma_categorie")

Un peu de documentation:

- https://docs.moodle.org/3x/fr/Format_XML_Moodle
- https://docs.moodle.org/3x/fr/Questions
- https://docs.moodle.org/31/en/Embedded_files_repository
- AMC to Moodle : https://github.com/nennigb/amc2moodle
- GIFT to Moodle : https://www.lmspulse.com/2020/gift-format-moodle-quiz-scripting/
- Dict to XML : https://stackoverflow.com/questions/36021526/converting-an-array-dict-to-xml-in-python
    - bon ca marche pas terrible relou avec attribute et plein de trucs automatiques
- Dict to XML : https://github.com/martinblech/xmltodict
    - meilleur controle sur tout
    - par contre l'output est un string avec des tabs? Ok facile a resoudre
    - ne gère pas le CDATA : besoin de https://github.com/martinblech/xmltodict/issues/57 ? Yes